In [ ]:

def get_GPT_answer(text, question):

  prompt_0 = """
I will provide you with a passage of text and a question . Based on the
type of question , please follow the steps below to provide your
answer . Determine the type of question according to the beginning of
the question , and answer the following questions according to the
different types :
1. For quantitative questions (" what is ") , you should provide a
corresponding number based on the information in the text . If the
text uses expressions such as "not less " or " more than " to represent
quantities , the original representation is used . If the text does
not mention any relevant data , respond with "No Answer ".
2. For qualitative questions (" Is there any description ") , when
answering "Is there any description " questions , ensure you carefully
verify the information . If the text clearly lacks the required
information , answer "no ". Otherwise , answer "yes ".
"""
  text_1 = "Passage: \n" + str(text)
  question_1 = "\n Question: \n" + question
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": prompt_0},
      {"role": "user", "content": text_1},
      {"role": "user", "content": question_1}
    ],
    temperature=0
  )

  return completion.choices[0].message.content



In [ ]:
def getMoldeAnswer(text, question):

  alpaca_prompt = """
I will provide you with a passage of text and a question . Based on the
type of question , please follow the steps below to provide your
answer . Determine the type of question according to the beginning of
the question , and answer the following questions according to the
different types :
1. For quantitative questions (" what is ") , you should provide a
corresponding number based on the information in the text . If the
text uses expressions such as "not less " or " more than " to represent
quantities , the original representation is used . If the text does
not mention any relevant data , respond with "No Answer ".
2. For qualitative questions (" Is there any description ") , when
answering "Is there any description " questions , ensure you carefully
verify the information . If the text clearly lacks the required
information , answer "no ". Otherwise , answer "yes ".
### Text:
{}

### Quetsion: :
{}

### Answer:
{}"""
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          text, # instruction
          question, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

  return_str = tokenizer.batch_decode(outputs)

  text = return_str[0]
  start_index = text.find("### Answer:")
  if start_index != -1:
    start_index = start_index  + len("### Answer:")
    result = text[start_index:]
  else:
    result = "No Answer"

  finish_index = result.find("### Quetsion:")
  if finish_index != -1:
    #start_index = start_index  + len("### Answer:")
    result = result[:finish_index]
  return result.lstrip()

In [ ]:
import time
start_time = time.time()
modle_Answer = []
original_Information = []
question = []
gold_Answer = []
question_Type = []

for index, row in df_QA_data.iterrows():
    print(index)
    original_Information.append(row['original_Information'])
    question.append(row['question'])
    gold_Answer.append(row['gold_Answer'])
    #change this function
    model_Answer_get = getMoldeAnswer(row['original_Information'],row['question'])

    modle_Answer.append(model_Answer_get)
finish_time = time.time()
print(finish_time - start_time)

In [ ]:
qa_sample = {'original_Information':original_Information,
         'question':question,
            'gold_Answer':gold_Answer,
             'modle_Answer':modle_Answer}

df_qa_sample_ss = pd.DataFrame(qa_sample)
fp = 'file_name.xlsx'
df_qa_sample_ss.to_excel(fp, index=False)